<a href="https://colab.research.google.com/github/BenShieh233/Learn_Python/blob/main/ExternalDataLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 5.5 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [2]:
import pandas as pd

#Q4 Template
I've helped you get started a little below

In [47]:
#Q4 import constraint coefficient data
constraint_df = pd.read_excel("/content/sample_data/q4data.xlsx", sheet_name = "constraintdata")
constraint_df

,coef_x1,coef_x2,coef_x3,coef_x4,coef_x5,constraint_rhs
0,3,2,-7,5,6,47
1,4,-8,-3,2,4,53
2,-3,4,6,-1,0,91


In [48]:
#Q4 import objective coefficient data
obj_df = pd.read_excel("/content/sample_data/q4data.xlsx", sheet_name = "objdata")
obj_df

,coef_x1,coef_x2,coef_x3,coef_x4,coef_x5,obj_constant
0,6,4,-7,3,2,0


In [49]:
#Q4 create list objects
obj_coef = obj_df.iloc[0,0:5].values.tolist()
const_coef = constraint_df.iloc[:,0:5].values.tolist()
const_rhs = constraint_df.iloc[:,5].values.tolist()

In [50]:
#Q5 create optimization problem
num_DVs = len(obj_coef)
num_const = len(const_rhs)
#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x = Var(range(num_DVs), domain = NonNegativeReals)

#define the objective
model.Objective = Objective(expr = sum(obj_coef[i]*model.x[i] for i in range(num_DVs)), sense = maximize)

#specify the constraints
model.constraints = ConstraintList()

for j in range(num_const):
  model.constraints.add(expr = sum(const_coef[j][i]*model.x[i] for i in range(num_DVs)) <= const_rhs[j])

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 6*x[0] + 4*x[1] - 7*x[2] + 3*x[3] + 2*x[4]

1 Co

In [52]:
#Q5 solve model
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')

results = opt.solve(model, tee = False) 

In [46]:
#Q5 print relevant results
print("obj*=", model.Objective())
for i in range(num_DVs):
  print(f"x{i} = {model.x[i]()}")

obj*= 456.1219512195104
x0 = 115.292682926829
x1 = 31.6219512195122
x2 = 51.7317073170732
x3 = 0.0
x4 = 0.0
